# Chapter 4. 신경망 시작하기: 분류와 회귀

__분류와 회귀에서 사용하는 용어__  
- **sample** 또는 **input**: 모델에 주입될 data point
- **prediction** 또는 **output**: 모델로부터 나오는 값
- **target**: 예측해야 하는 정답
- **prediction error** 또는 **loss**: 모델의 예측과 정답 사이의 거리를 측정한 값
- **class**: 분류 문제에서 선택가능한 레이블의 집합
- **label**: 분류 문제에서 샘플에 할당된 클래스명
- **ground-truth** 또는 **annotation**: 일반적으로 사람에 의해 수집되는 데이터셋의 타깃
- **binary classification**: 각 입력 샘플을 2개의 서로 다른 클래스로 구분하는 작업
- **multiclass classification**: 각 입력 샘플을 2개 이상의 서로 다른 클래스로 구분하는 작업
- **multi-label classification**: 각 입력 샘플을 여러 개의 레이블에 할당하며 구분하는 작업
- **scalar regression**: 회귀하는 스칼라 값을 찾는 작업
- **vector regression**: 회귀하는 벡터 값을 찾는 작업
- **mini-batch** 또는 **batch**: 모델에 의해 동시에 처리되는 샘플의 묶음

## 4.1 영화 리뷰 분류: 이진 분류 문제

### 4.1.1 IMDB 데이터셋

In [5]:
# IMDB dataset load
from tensorflow import keras
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

positive = 1, negative = 0

In [8]:
# review decoding
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# 0 = padding, 1 = start, 2 = unknown으로 예약되어 있으므로 3을 빼준다.
decoded_review = " ".join([reverse_word_index.get(i - 3, "?") for i in train_data[0]])
print(decoded_review)

? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

## 4.1.2 데이터 준비

신경망에는 동일한 길이의 입력들을 주어야 한다.  
숫자 리스트는 다음과 같은 방법으로 텐서로 변환한다.
- 리스트에 padding을 추가하여 (samples, max_length) 크기의 정수 텐서로 변환한다.  
- 리스트를 multi-hot encoding하여 0과 1의 벡터로 변환한다.  
  (samples, num_words) 크기의 정수 텐서로 변환된다.

In [9]:
# multi-hot encoding
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [10]:
x_train[0]

array([0., 1., 1., ..., 0., 0., 0.])

In [11]:
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")
y_train[0]

1.0